### Welcome to the exercise about observational studies! This exercise will be hands on, and you will be able to practise the skills you developed so far!

## Propensity score matching

In this exercise, you will apply [propensity score matching](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf), which we discussed in lecture 6 ("Observational studies"), in order to draw conclusions from an observational study.

We will work with a by-now classic dataset from Robert LaLonde's study "[Evaluating the Econometric Evaluations of Training Programs](http://people.hbs.edu/nashraf/LaLonde_1986.pdf)" (1986).
The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program.
Your task is to determine the effectiveness of the "treatment" represented by the job training program.

#### Dataset description

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)

If you want to deepen your knowledge on propensity scores and observational studies, we highly recommend Rosenbaum's excellent book on the ["Design of Observational Studies"](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf). Even just reading the first chapter (18 pages) will help you a lot.

In [9]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns
import networkx as nx
%matplotlib inline

data_folder = './data/'
df = pd.read_csv(data_folder + 'lalonde.csv')

In [3]:
df.sample(10)

,id,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
148,NSW149,1,26,11,1,0,1,1,0.0000,2754.6460,26372.28000
379,PSID195,0,32,4,0,0,1,1,0.0000,1378.5480,0.00000
380,PSID196,0,18,11,1,0,0,1,0.0000,1367.8060,33.98771
555,PSID371,0,26,14,0,0,0,0,0.0000,0.0000,6717.74500
451,PSID267,0,21,14,0,0,0,0,107.7597,293.6129,7698.95500
604,PSID420,0,39,2,1,0,1,1,0.0000,0.0000,964.95550
437,PSID253,0,21,9,1,0,0,1,1030.5740,470.8548,1223.55800
143,NSW144,1,46,8,1,0,0,1,3165.6580,2594.7230,0.00000
495,PSID311,0,55,7,0,0,1,1,8832.3750,0.0000,0.00000
115,NSW116,1,21,12,0,0,0,0,3670.8720,334.0494,12558.02000


## 1. A naive analysis

Compare the distribution of the outcome variable (`re78`) between the two groups, using plots and numbers.
To summarize and compare the distributions, you may use the techniques we discussed in lecture 4 ("Descibing data") and 3 ("Visualizing data").

What might a naive "researcher" conclude from this superficial analysis?

In [4]:
''' your code and explanations ''';

## 2. A closer look at the data

You're not naive, of course (and even if you are, you've learned certain things in ADA), so you aren't content with a superficial analysis such as the above.
You're aware of the dangers of observational studies, so you take a closer look at the data before jumping to conclusions.

For each feature in the dataset, compare its distribution in the treated group with its distribution in the control group, using plots and numbers.
As above, you may use the techniques we discussed in class for summarizing and comparing the distributions.

What do you observe?
Describe what your observations mean for the conclusions drawn by the naive "researcher" from his superficial analysis.

In [5]:
''' your code and explanations ''';

## 3. A propensity score model

Use logistic regression to estimate propensity scores for all points in the dataset.
You may use `statsmodels` to fit the logistic regression model and apply it to each data point to obtain propensity scores.

Recall that the propensity score of a data point represents its probability of receiving the treatment, based on its pre-treatment features (in this case, age, education, pre-treatment income, etc.).
To brush up on propensity scores, you may read chapter 3.3 of the above-cited book by Rosenbaum or [this article](https://drive.google.com/file/d/0B4jctQY-uqhzTlpBaTBJRTJFVFE/view).

Note: you do not need a train/test split here. Train and apply the model on the entire dataset. If you're wondering why this is the right thing to do in this situation, recall that the propensity score model is not used in order to make predictions about unseen data. Its sole purpose is to balance the dataset across treatment groups.
(See p. 74 of Rosenbaum's book for an explanation why slight overfitting is even good for propensity scores.
If you want even more information, read [this article](https://drive.google.com/file/d/0B4jctQY-uqhzTlpBaTBJRTJFVFE/view).)

In [11]:
# let's standardize the continuous features
df['age'] = (df['age'] - df['age'].mean())/df['age'].std()
df['educ'] = (df['educ'] - df['educ'].mean())/df['educ'].std()
df['re74'] = (df['re74'] - df['re74'].mean())/df['re74'].std()
df['re75'] = (df['re75'] - df['re75'].mean())/df['re75'].std()

mod = smf.logit(formula='treat ~ age + educ + C(black) + C(hispan)  + C(married) + C(nodegree) + +re74 + re75', data=df)

res = mod.fit()

# Extract the estimated propensity scores
df['Propensity_score'] = res.predict()

print(res.summary())

Optimization terminated successfully.
         Current function value: 0.397267
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  treat   No. Observations:                  614
Model:                          Logit   Df Residuals:                      605
Method:                           MLE   Df Model:                            8
Date:                Fri, 08 Jan 2021   Pseudo R-squ.:                  0.3508
Time:                        19:46:43   Log-Likelihood:                -243.92
converged:                       True   LL-Null:                       -375.75
Covariance Type:            nonrobust   LLR p-value:                 2.194e-52
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -2.8509      0.350     -8.147      0.000      -3.537      -2.165
C(black)[T.

## 4. Balancing the dataset via matching

Use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group.
(Hint: you may explore the `networkx` package in Python for predefined matching functions.)

Your matching should maximize the similarity between matched subjects, as captured by their propensity scores.
In other words, the sum (over all matched pairs) of absolute propensity-score differences between the two matched subjects should be minimized.

This is how networkx library can help you do this. Each possible pair of (treated_person, control_person) is characterized by a similarity. This is how we can initialize a graph, and add an edge for one possible pair. We then need to add an edge for each possible pair.
    - G = nx.Graph()
    - G.add_weighted_edges_from([(control_person, treated_person, similarity)])
Optimal matching is then found with:
    - matching = nx.max_weight_matching(G)

After matching, you have as many treated as you have control subjects.
Compare the outcomes (`re78`) between the two groups (treated and control).

Also, compare again the feature-value distributions between the two groups, as you've done in part 2 above, but now only for the matched subjects.
What do you observe?
Are you closer to being able to draw valid conclusions now than you were before?

In [12]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [28]:
# Separate the treatment and control groups
treatment_df = df[df['treat'] == 1]
control_df = df[df['treat'] == 0]

# Create an empty undirected graph
G = nx.Graph()

# Loop through all the pairs of instances
for control_id, control_row in control_df.iterrows():
    for treatment_id, treatment_row in treatment_df.iterrows():

        # Calculate the similarity 
        similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])

        # Add an edge between the two instances weighted by the similarity between them
        G.add_weighted_edges_from([(control_id, treatment_id, similarity)])

# Generate and return the maximum weight matching on the generated graph
matching = nx.max_weight_matching(G)

In [29]:
matching = np.array(list(matching))
balanced_df_1 = df.iloc[matching.flatten()]
balanced_df_1.describe()
# race not balanced

,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78,Propensity_score
count,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000
mean,0.500000,-0.182542,0.078737,0.656757,0.137838,0.200000,0.672973,-0.361025,-0.185558,5901.959790,0.470163
std,0.500677,0.913502,0.896839,0.475435,0.345197,0.400542,0.469762,0.705428,0.891102,7028.505472,0.260227
min,0.000000,-1.149982,-3.526478,0.000000,0.000000,0.000000,0.000000,-0.703546,-0.662971,0.000000,0.024896
25%,0.000000,-0.846375,-0.482714,0.000000,0.000000,0.000000,0.000000,-0.703546,-0.662971,31.771122,0.177684
50%,0.500000,-0.441566,0.278227,1.000000,0.000000,0.000000,1.000000,-0.703546,-0.654822,3886.808000,0.581370
75%,1.000000,0.165649,0.658697,1.000000,0.000000,0.000000,1.000000,-0.402532,-0.089452,9082.846000,0.697506
max,1.000000,2.796912,2.941520,1.000000,1.000000,1.000000,1.000000,4.705571,6.965879,60307.930000,0.853153


## 5. Balancing the groups further

Based on your comparison of feature-value distributions from part 4, are you fully satisfied with your matching?
Would you say your dataset is sufficiently balanced?
If not, in what ways could the "balanced" dataset you have obtained still not allow you to draw valid conclusions?

Improve your matching by explicitly making sure that you match only subjects that have the same value for the problematic feature.
Argue with numbers and plots that the two groups (treated and control) are now better balanced than after part 4.


In [36]:
# Create an empty undirected graph
G = nx.Graph()

# Loop through all the pairs of instances
for control_id, control_row in control_df.iterrows():
    for treatment_id, treatment_row in treatment_df.iterrows():

        # Calculate the similarity 
        if control_row['black'] == treatment_row['black'] and \
            control_row['hispan'] == treatment_row['hispan']:
            similarity = get_similarity(control_row['Propensity_score'],
                                        treatment_row['Propensity_score'])

        # Add an edge between the two instances weighted by the similarity between them
        G.add_weighted_edges_from([(control_id, treatment_id, similarity)])

# Generate and return the maximum weight matching on the generated graph
matching2 = nx.max_weight_matching(G)

In [37]:
matching2 = np.array(list(matching2))
balanced_df_2 = df.iloc[matching2.flatten()]
balanced_df_2.describe()

,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78,Propensity_score
count,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000
mean,0.500000,-0.177619,0.073595,0.605405,0.083784,0.202703,0.656757,-0.343267,-0.187089,5686.315600,0.439029
std,0.500677,0.915945,0.864701,0.489425,0.277438,0.402557,0.475435,0.697757,0.874578,6858.134154,0.285185
min,0.000000,-1.149982,-3.526478,0.000000,0.000000,0.000000,0.000000,-0.703546,-0.662971,0.000000,0.015487
25%,0.000000,-0.846375,-0.482714,0.000000,0.000000,0.000000,0.000000,-0.703546,-0.662971,39.159753,0.104865
50%,0.500000,-0.441566,0.278227,1.000000,0.000000,0.000000,1.000000,-0.703546,-0.614819,3747.122000,0.569317
75%,1.000000,0.165649,0.658697,1.000000,0.000000,0.000000,1.000000,-0.296844,0.013151,8895.676500,0.695175
max,1.000000,2.796912,2.941520,1.000000,1.000000,1.000000,1.000000,4.705571,6.965879,60307.930000,0.853153


## 6. A less naive analysis

Compare the outcomes (`re78`) between treated and control subjects, as you've done in part 1, but now only for the matched dataset you've obtained from part 5.
What do you conclude about the effectiveness of the job training program?

In [9]:
''' your code and explanations ''';